In [1]:
import numpy as np
import tensorflow as tf

In [2]:
data_train = np.load('C:/Users/User/PycharmProjects/AIproject2/인프데이터/data_train.npy')
labels_train = np.load('C:/Users/User/PycharmProjects/AIproject2/인프데이터/labels_train.npy')
data_test = np.load('C:/Users/User/PycharmProjects/AIproject2/인프데이터/data_test.npy')
labels_test = np.load('C:/Users/User/PycharmProjects/AIproject2/인프데이터/labels_test.npy')

In [3]:
np.shape(data_train)

(1440, 30, 129)

In [4]:
data_train_num = np.shape(data_train)[0]

In [15]:
tf.reset_default_graph()

In [16]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [14]:
X = tf.placeholder(tf.float32, [None, 30, 129, 1])
Y = tf.placeholder(tf.float32, [None, 6])
keep_prob=tf.placeholder(tf.float32)

W1 = tf.get_variable('W1', shape=[3,3,1,32], initializer=tf.contrib.layers.xavier_initializer())
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W2 = tf.get_variable('W2', shape=[3,3,32,64], initializer=tf.contrib.layers.xavier_initializer())
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W3 = tf.get_variable('W3', shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
L3 = tf.reshape(L2, [-1,7*7*64])
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3,keep_prob)

W4 = tf.get_variable('W4', shape=[256,6], initializer=tf.contrib.layers.xavier_initializer())
model = tf.matmul(L3,W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost, global_step=global_step)

ValueError: Variable W1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


In [17]:
X = tf.placeholder(tf.float32, [None, 30, 129, 1])
Y = tf.placeholder(tf.float32, [None, 6])
is_training = tf.placeholder(tf.bool)

L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(X, 64, [3, 3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 200, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 6, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost, global_step=global_step)

In [18]:
init = tf.global_variables_initializer()
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('C:/Users/User/PycharmProjects/AIproject2/model')

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(init)

batch_size = 120
total_batch = int(data_train_num/batch_size)

for epoch in range(100):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys= data_train[i*batch_size:i*batch_size+batch_size], labels_train[i*batch_size:i*batch_size+batch_size]
        batch_xs = batch_xs.reshape(-1, 30, 129, 1)
        _,cost_val = sess.run([optimizer, cost],
                             feed_dict={X: batch_xs,
                                        Y: batch_ys})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:4f}'.format(total_cost / total_batch))
    
print('최적화 완료')
saver.save(sess, 'C:/Users/User/PycharmProjects/AIproject2/model/voice.ckpt', global_step=global_step)

Epoch: 0001 Avg. cost = 967.040780
Epoch: 0002 Avg. cost = 155.219363
Epoch: 0003 Avg. cost = 37.769378
Epoch: 0004 Avg. cost = 12.254823
Epoch: 0005 Avg. cost = 6.080712
Epoch: 0006 Avg. cost = 2.227051
Epoch: 0007 Avg. cost = 1.024510
Epoch: 0008 Avg. cost = 0.823066
Epoch: 0009 Avg. cost = 0.323245
Epoch: 0010 Avg. cost = 0.320254
Epoch: 0011 Avg. cost = 0.246587
Epoch: 0012 Avg. cost = 0.224707
Epoch: 0013 Avg. cost = 0.211338
Epoch: 0014 Avg. cost = 0.180740
Epoch: 0015 Avg. cost = 0.176203
Epoch: 0016 Avg. cost = 0.152861
Epoch: 0017 Avg. cost = 0.142357
Epoch: 0018 Avg. cost = 0.127260
Epoch: 0019 Avg. cost = 0.117834
Epoch: 0020 Avg. cost = 0.106300
Epoch: 0021 Avg. cost = 0.100059
Epoch: 0022 Avg. cost = 0.114727
Epoch: 0023 Avg. cost = 0.090281
Epoch: 0024 Avg. cost = 0.080057
Epoch: 0025 Avg. cost = 0.144538
Epoch: 0026 Avg. cost = 0.105412
Epoch: 0027 Avg. cost = 0.116381
Epoch: 0028 Avg. cost = 0.103028
Epoch: 0029 Avg. cost = 0.117373
Epoch: 0030 Avg. cost = 0.085552
Epoc

'C:/Users/User/PycharmProjects/AIproject2/model/voice.ckpt-1200'

In [19]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict={X: data_test.reshape(-1, 30, 129, 1),
                                                Y: labels_test,
                                                is_training: False}))
prediction = tf.argmax(model,1)
target = tf.argmax(Y, 1)
print('예측값: ', sess.run(prediction, feed_dict={X: data_test.reshape(-1, 30, 129, 1),
                                                  Y: labels_test,
                                                  is_training: False}))
print('실제값: ', sess.run(target, feed_dict={X: data_test.reshape(-1, 30, 129, 1),
                                                  Y: labels_test,
                                                  is_training: False}))

정확도:  0.833333
예측값:  [0 1 2 3 2 5]
실제값:  [0 1 2 3 4 5]


In [21]:
sess.close()